# Improved SCA based on SVM
---

In this notebook, we will set up the hardware and capture power traces as we did in `0_capture_training_traces` \
If you don't have a Chipwhisperer Nano you can skip this part and use the included traces.

---
#### 1. Connecting the device, building and flashing the firmware

In [2]:
import chipwhisperer as cw

In [5]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_2_1'

In [6]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [7]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
make[1]: '.dep' is up to date.
Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
arm-none-eabi-gcc (GNU Arm Embedded Toolchain 10-2020-q4-major) 10.2.1 20201103 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Welcome to another exciting ChipWhisperer target build!!
.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF_C

In [9]:
cw.program_target(scope, prog, "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))


Detected known STMF32: STM32F03xx4/03xx6
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5859 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5859 bytes


#### 2. Setting the encryption key and capturing the power traces
We will use the **same plaintexts** as in the training traces with a **different key**.

In [10]:
import numpy as np

project_folder = 'SCA_SVM_DATA'    
traces_folder = f'{project_folder}/traces'
project_file = f'{traces_folder}/training_traces.cwp'

project = cw.open_project(project_file)
n = 10000
textins = project.textins[n:]
print(f"✅ Loaded {len(textins)} plaintexts")
project.close()

✅ Loaded 10000 plaintexts


In [12]:
key = bytes.fromhex("f47e151628aed2a6abf7158809cf4f3c")
target.set_key(key)
print(f"Key set to: {key.hex()}")

Key set to: f47e151628aed2a6abf7158809cf4f3c


In [13]:
from tqdm.notebook import trange

traces = []
textouts = []

N = 10000

# Capture traces in a loop with a progress bar.
for i in trange(N, desc='Capturing traces'):
    scope.arm()  # Arm the scope.
    
    # Send the current plaintext to the target.
    target.simpleserial_write('p', textins[i])
    
    # Capture the power trace.
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue  # Skip this trace if capture fails.
    
    # Read the target's response.
    response = target.simpleserial_read('r', 16)
    
    # Save the captured trace, plaintext, and key.
    traces.append(scope.get_last_trace())
    textouts.append(response)

# Convert the list of traces to a NumPy array for easier processing.
traces = np.array(traces)
textouts = np.array(textouts)
print(f"Captured {len(traces)} traces.")

Capturing traces:   0%|          | 0/10000 [00:00<?, ?it/s]

Captured 10000 traces.


#### 3. Saving the traces and disconnecting the hardware

In [14]:
project_file = f'{traces_folder}/attack_traces.cwp'
project = cw.create_project(project_file, overwrite = True)
for i, (trace, textout) in enumerate(zip(traces, textouts)):
    project.traces.append(cw.Trace(wave=trace, textin=textins[i], textout=textout, key=key))
project.save()

print(f"{len(traces)} traces saved to {project_file}")

10000 traces saved to SCA_SVM_DATA/traces/attack_traces.cwp


In [15]:
scope.dis()
target.dis()

These traces will be fed into the models to retrieve the encryption key.